**Proyecto en grupo (2 miembros) - opción A:** El proyecto en grupo consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia según la taxonomía siguiente: 

- 1- Mundo, 2- Economía, 3- Política y Conflictos, 4- Ciencias y Tecnología, 5- Catástrofes y Accidentes, 6- Cultura y Artes, 7- Deporte, 8- Ecología y Planeta, 9- Crimen, delitos y Justicia, 10- Salud

Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

In [1]:
import re

linku = 'https://www.emol.com/noticias/Economia/2021/03/01/1013618/Impuesto-ultra-millonarios-Estados-Unidos.html'
linku.split('/')

['https:',
 '',
 'www.emol.com',
 'noticias',
 'Economia',
 '2021',
 '03',
 '01',
 '1013618',
 'Impuesto-ultra-millonarios-Estados-Unidos.html']

In [2]:
def get_tag(row):
    if row.media_outlet == 'emol':
        tag_idx = 4
    else:
        tag_idx = 3
    link = row.url.split('/')
    if link[tag_idx] == 'noticias' and link[tag_idx + 1] not in ['2020', '2021']:
        return link[tag_idx + 1]
    return link[tag_idx]

def clean_df(df):
    df.drop('year', inplace=True, axis=1)
    df.drop('id_journalist', inplace=True, axis=1)
    #print(corpus['media_outlet'].unique())
    df['tag'] = df.apply(get_tag, axis=1)
    df['text'] = df['text'].astype('string')
    df = df[df['text'].notna()]
    return df

In [3]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm

unique_tags = set()
folder = './noticias'
dfs = []
for archivo in tqdm(listdir(folder)):
    corpus = pd.read_csv(join(folder, archivo))
    corpus = clean_df(corpus)
    dfs.append(corpus)
    #print(corpus.head())
    #print(archivo, corpus.tag.unique())
    for tag in corpus.tag.unique():
        unique_tags.add(tag)

100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.31it/s]


In [4]:
print(len(unique_tags))
for tag in unique_tags:
    print(tag)

84
artes
tv
Espectaculos
pulso
mexico
trabajo
mundo
calidad-de-vida
Autos
columnas
tendencias
la-tercera-pm
eclipse-2020
chile
dia
data
genero
pais
peru
Tecnologia
la-tercera-tv
especiales
Internacional
culto
turismo
noticiasbbc
programas
tecnologia-2
negocios
multimedia
pulso-trader
redes-sociales
masdeco
Tendencias
que-pasa
cultura
el-semanal
proceso-constituyente
sin-editar
sociedad
ciencia-tecnologia
portada
plebiscito-chile-elige
pueblos
opinion
primarias-2020
regiones
noticias
editorial
reconstitucion
mouse
economia
internacional
la-tercera-domingo
podcast
tesirve
actualidad
el-deportivo
cartas-ciudadanas
reportaje-investigacion
coronavirus
braga
latinoamerica
wires
ciudadanos-al-poder
medio-ambiente
Deportes
justicia
politica
animal
mapuche
nacional
paula
salud
deportes
mineria
Economia
mercados
Nacional
revista-que-pasa
destacado
educacion
derechos-humanos
conversacioneslt


In [5]:
dm = pd.read_csv('./categorias - hojav2.csv')
dm.rename(columns={'Unnamed: 0':'tag'}, inplace=True)
vals = dm.iloc[:, 1:]
labels = vals[vals.max(axis=1)>0].idxmax(axis=1)

In [6]:
cats = pd.DataFrame(labels)
cats = cats.join(dm['tag'])
cats.rename(columns={0:'cat'}, inplace=True)
cats_dict = dict(zip(cats['tag'], cats['cat']))
cats_dict

{'animal': '8-Ecología_y_Planeta',
 'artes': '6-Cultura_y_Artes',
 'Autos': '4-Ciencias_y_Tecnología',
 'braga': '3-Política_y_Conflictos',
 'ciencia-tecnologia': '4-Ciencias_y_Tecnología',
 'coronavirus': '10-Salud',
 'culto': '6-Cultura_y_Artes',
 'cultura': '6-Cultura_y_Artes',
 'Deportes': '7-Deporte',
 'deportes': '7-Deporte',
 'eclipse-2020': '8-Ecología_y_Planeta',
 'Economia': '2-Economía',
 'economia': '2-Economía',
 'el-deportivo': '7-Deporte',
 'Espectaculos': '6-Cultura_y_Artes',
 'Internacional': '1-Mundo',
 'internacional': '1-Mundo',
 'justicia': '9-Crimen_delitos_y_Justicia',
 'la-tercera-domingo': '6-Cultura_y_Artes',
 'la-tercera-tv': '2-Economía',
 'latinoamerica': '1-Mundo',
 'mapuche': '3-Política_y_Conflictos',
 'medio-ambiente': '8-Ecología_y_Planeta',
 'mercados': '2-Economía',
 'mexico': '1-Mundo',
 'mineria': '3-Política_y_Conflictos',
 'mouse': '4-Ciencias_y_Tecnología',
 'multimedia': '4-Ciencias_y_Tecnología',
 'mundo': '1-Mundo',
 'negocios': '9-Crimen_del

In [7]:
import numpy as np

final_df = pd.concat(dfs)
final_df.drop_duplicates(subset='url', keep='first', inplace=True)
final_df['category'] = final_df.apply(lambda row : cats_dict.get(row.tag, np.nan), axis=1)
#id se puede borraar
final_df = final_df[['id','text', 'category']]
final_df = final_df[final_df['category'].notna()]
final_df

,id,text,category
14,13115,"Este martes, el Presidente Sebastián Piñera a...",10-Salud
18,13183,La aduana china detectó la presencia del virus...,10-Salud
125,614680,"El presidente de México, Andrés Manuel López O...",1-Mundo
126,614690,"El presidente de Uruguay, Luis Lacalle Pou, an...",1-Mundo
127,614700,"El presidente boliviano, Luis Arce, destituyó ...",1-Mundo
...,...,...,...
2652,17781918,Con la era de los eVTOL (vehículos eléctricos ...,4-Ciencias_y_Tecnología
2653,17781934,El famoso piloto Ken Block decidió llevar a ...,4-Ciencias_y_Tecnología
2654,17781979,El fabricante de autos japonés Toyota celebra ...,4-Ciencias_y_Tecnología
2655,17781982,La temporada 2021 del Copec RallyMobil inició ...,4-Ciencias_y_Tecnología


In [8]:
pd.set_option('display.max_colwidth', 80)
print(final_df['text'].loc[final_df['id'] == 5688227])

4366    La segunda tanda de la vacuna rusa Sputnik V contra el coronavirus, arribó e...
Name: text, dtype: string


In [9]:
from pandasql import sqldf

q="""SELECT category, count(*) FROM final_df GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8629
1,7-Deporte,6924
2,4-Ciencias_y_Tecnología,4344
3,2-Economía,2637
4,3-Política_y_Conflictos,1934
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


In [10]:
text = final_df.sample(n=1)['text'].values[0]
text

'A diez días de haber regresado a Argentina y tras mantener un perfil muy bajo durante los últimos nueve meses, el  expresidente Mauricio Macri  rompió el silencio y  realizó un análisis de la gestión del actual Mandatario, Alberto Fernández, en muy duros términos . " Las autoridades al frente del Poder Ejecutivo vienen desplegando una serie de medidas que consisten en el ataque sistemático y permanente a nuestra Constitución. Para poder gobernar sin límites , violentan la ley fundamental de la nación", escribió el exmandatario en una columna publicada en "La Nación".  "Esta sociedad ya no tolera el atropello a las instituciones, la corrupción de los funcionarios, el delito en ninguna forma, la injusticia, la inseguridad, la impunidad, la violencia, la prepotencia y la anomia" Mauricio Macri, expresidente de Argentina " Se vulnera el pleno funcionamiento de la República, porque no se respetan los reglamentos de las cámaras del Congreso para debatir y sancionar las leyes . Se pretende c

In [11]:
# Word tokenization
from spacy.lang.es import Spanish

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = Spanish()

text = final_df.sample(n=1)['text'].values[0][:100]
#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

['El', 'magnate', 'hongkonés', 'de', 'la', 'prensa', ' ', 'Jimmy', 'Lai', ' ', 'fue', 'hoy', 'condenado', 'a', ' ', 'un', 'año', 'de', 'prisión', 'por', 'organizar', 'y']


In [12]:
#Stop words
#importing stop words from English language.
from spacy.lang.es.stop_words import STOP_WORDS

#Implementation of stop words:
filtered_sent=[]

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

for word in doc:
    if word.is_stop==False:
        filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

Filtered Sentence: [magnate, hongkonés, prensa,  , Jimmy, Lai,  , condenado, a,  , año, prisión, organizar, y]


In [13]:
# !python -m spacy download es_core_news_md

In [14]:
import string
import spacy

nlp = spacy.load('es_core_news_md', disable=['parser','ner','textcat','...'] )
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load("es_core_news_md")
stop_words = STOP_WORDS

In [15]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

# Creating our tokenizer function
def spacy_tokenizer(sentence, stem=True):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens]
    
    # Removing stop words, ver lo de puntuuacion con  token.tag_
    mytokens = [ word for word in mytokens 
                          if word not in stop_words 
                          and word not in punctuations ]
    if stem:
        mytokens = [ stemmer.stem(word) for word in mytokens ]

    # return preprocessed list of tokens
    return mytokens

In [16]:
text = final_df.sample(n=1)['text'].values[0][:100]
print(text)
spacy_tokenizer(text)

A las pocas horas de jurar como presidente este miércoles, Joe Biden enviará un proyecto de ley que 


['a',
 'hor',
 'jur',
 'president',
 'miercol',
 'joe',
 'bid',
 'envi',
 'proyect',
 'ley']

In [17]:
doc = nlp(text[:20])
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
           token.shape_, token.is_alpha, token.is_stop)

A a ADP ADP case X True False
las el DET DET det xxx True True
pocas poco DET DET det xxxx True True
horas hora NOUN NOUN ROOT xxxx True True
de de ADP ADP punct xx True True
